## Plot CRT On Board Survey frequency

```
OMX conversion Voyager syntax: 
CONVERTMAT FROM='skm_d8_Pk.mtx', TO='skm_d8_Pk.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_d8_Ok.mtx', TO='skm_d8_Ok.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_w8_Pk.mtx', TO='skm_w8_Pk.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_w8_Ok.mtx', TO='skm_w8_Ok.omx', FORMAT=OMX COMPRESSION=6
```

In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets

In [41]:
model_paths = {
    "RTP v9.0"                            : r"\\modelace\ModelAce-E\1 - TDM\1 - Official Release (full run)\v9x\v9.1\v9.1.0\WF-TDM-v9.1.0 - official\Scenarios\BY_2019",
    "TDM-Recalib"                         : "../Scenarios/v920-E2.14.1/BY_2019",
    "TDM-Recalib-100-divisor-removed"     : "../Scenarios/v920-E2.14.2/BY_2019",
    "TDM-Recalib-second-part-ivt-removed" : "../Scenarios/v920-E2.14.3/BY_2019",
    "Single TLF-RunFac-2.5"               : "../Scenarios/v920-E2.13.2/BY_2019",
    "Purpose TLF-RunFac-2.5"              : "../Scenarios/v920-E2.13.3/BY_2019",
    "Purpose/Period TLF-RunFac-2.5"       : "../Scenarios/v920-E2.13.4/BY_2019"
}

obs_distance_model = "TDM-Recalib-second-part-ivt-removed"

In [43]:
# Define transit skim files
transit_skim_loc = [
    r"1a_Skims\skm_d8_Pk.omx", r"1a_Skims\skm_d8_Ok.omx",
    r"1a_Skims\skm_w8_Pk.omx", r"1a_Skims\skm_w8_Ok.omx"
]

# Define invalid TAZs
invalid_model_taz_list = np.arange(3547, 3601)

# Define purpose, periods, and access modes
purposes = ['HBW', 'HBO', 'NHB', 'HBC']
periods = ['PK', 'OK']
accesses = ['dCRT', 'wCRT']

skimloc_mapping = {
    ('PK', 'dCRT'): 0,
    ('PK', 'wCRT'): 2,
    ('OK', 'dCRT'): 1,
    ('OK', 'wCRT'): 3
}
access_mapping = {'dCRT': 'Drive', 'wCRT': 'Walk'}

# Load OBS Data
OBS_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)

# Filter OBS Data
CRTpkok = OBS_df[OBS_df['Linked_Mode'] == 8]
columns = ['p_TAZID', 'a_TAZID', 'Ac_Mode_Model', 'PK_OK', 'linked_weight_adj', 'Veh_Cat3p', 'Purp5_text']
CRTpkok = CRTpkok[columns]
CRTpkok.loc[CRTpkok['Veh_Cat3p'] == 3, 'Veh_Cat3p'] = 2
CRTpkok.rename(columns={'linked_weight_adj':'trips_count'}, inplace=True)


# Function to convert model data into DataFrame
def create_model_mtx_to_df(trips_file_name, crtime_file_name, trips_mtx_name, crtime_mtx_name='D8', delZero=True):
    trips_file = omx.open_file(trips_file_name)
    trips_mtx = np.array(trips_file[trips_mtx_name])
    crtime_file = omx.open_file(crtime_file_name)
    crtime_mtx = np.array(crtime_file[crtime_mtx_name])

    trips_df = pd.DataFrame(pd.DataFrame(trips_mtx).stack()).rename({0: 'trips_count'}, axis=1)
    crtime_df = pd.DataFrame(pd.DataFrame(crtime_mtx).stack()).rename({0: 'cr_travel_distance'}, axis=1)

    model_df = pd.concat([trips_df, crtime_df], axis=1).reset_index().rename(
        {'level_0': 'p_TAZID', 'level_1': 'a_TAZID'}, axis=1)

    model_df['p_TAZID'] += 1
    model_df['a_TAZID'] += 1
    model_df = model_df[~model_df['p_TAZID'].isin(invalid_model_taz_list) & ~model_df['a_TAZID'].isin(invalid_model_taz_list)]

    # **Filter out rows where trips_count == 0**
    if delZero: 
        model_df = model_df[model_df['trips_count'] > 0]

    return model_df

# Dictionary to store results from all models
model_results = {}

# Iterate through model versions
for version, Model_path in model_paths.items():
    print("")
    print(f"Processing Model Version: {version}")
    model_data_loc = os.path.join(Model_path, r"4_ModeChoice\1a_Skims")
    model_data_loc2 = os.path.join(Model_path, r"4_ModeChoice")

    for purpose in purposes:
        print(f"...{purpose}", end="")
        for period in periods:
            print(f" {period}", end="")
            if purpose=='HBC' and period=='OK':
                print(f" skip", end="")
                continue
            for access in accesses:
                print(f" {access}", end="")

                access_mapped = access_mapping.get(access, 'Unknown')  # Returns 'Drive'

                model_df = create_model_mtx_to_df(
                    trips_file_name=os.path.join(model_data_loc2, '2_DetailedTripMatrices', f"{purpose}_trips_allsegs_{period}.omx"),
                    crtime_file_name=os.path.join(model_data_loc2, transit_skim_loc[skimloc_mapping.get((period, access), None)]),
                    trips_mtx_name=access,
                    delZero = True
                )

                model_df['trips_count'] /= 100
                model_df = model_df[model_df['cr_travel_distance'] > 0]

                key = f"{purpose}_{access}_{period}_{version}"
                model_results[key] = model_df.copy()

                if version == obs_distance_model:
                    obs_temp_df = create_model_mtx_to_df(
                        trips_file_name=os.path.join(model_data_loc2, '2_DetailedTripMatrices', f"{purpose}_trips_allsegs_{period}.omx"),
                        crtime_file_name=os.path.join(model_data_loc2, transit_skim_loc[skimloc_mapping.get((period, access), None)]),
                        trips_mtx_name=access,
                        delZero=False
                    )
                    obs_temp_df.drop(columns=['trips_count'], inplace=True)
                    obs_temp_df = obs_temp_df[obs_temp_df['cr_travel_distance'] > 0]

                    obs_trips_df = CRTpkok[(CRTpkok['Purp5_text']==purpose) & (CRTpkok['PK_OK']==period) & (CRTpkok['Ac_Mode_Model']==access_mapped)]
                    obs_trips_df = obs_trips_df[['p_TAZID','a_TAZID','trips_count']].copy()
                    df = pd.merge(obs_temp_df, obs_trips_df, on=['p_TAZID','a_TAZID'])

                    key = f"{purpose}_{access}_{period}_OBS"
                    model_results[key] = df.copy()

print("")
print("Done!")


Processing Model Version: RTP v9.0
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: TDM-Recalib
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: TDM-Recalib-100-divisor-removed
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: TDM-Recalib-second-part-ivt-removed
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: Single TLF-RunFac-2.5
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip
Processing Model Version: Purpose TLF-RunFac-2.5
...HBW PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT...NHB PK dCRT wCRT OK dC

In [5]:
df

,p_TAZID,a_TAZID,cr_travel_distance,trips_count
0,417,1051,44.26,4.224
1,507,1231,39.94,5.976
2,737,2848,60.92,3.395
3,780,1231,23.46,3.742
4,826,2848,47.42,3.395
5,870,2848,47.42,3.395
6,990,2848,39.52,3.789
7,1025,2848,39.52,7.485
8,1038,2939,43.98,2.526
9,1065,437,37.10,1.746


In [47]:
# Create first set of dropdowns
model_dropdown_1 = widgets.Dropdown(
    options=['OBS'] + list(model_paths.keys()),
    value=list(model_paths.keys())[0],
    description="Model 1:",
    layout=widgets.Layout(width="50%")
)

purpose_dropdown_1 = widgets.Dropdown(
    options=purposes,
    value=purposes[0],
    description="Purpose 1:"
)

period_dropdown_1 = widgets.Dropdown(
    options=periods,
    value=periods[0],
    description="Period 1:"
)

access_dropdown_1 = widgets.Dropdown(
    options=accesses,
    value=accesses[0],
    description="Access 1:"
)

# Create second set of dropdowns
model_dropdown_2 = widgets.Dropdown(
    options=['OBS'] + list(model_paths.keys()),
    value='OBS',
    description="Model 2:",
    layout=widgets.Layout(width="50%")
)

purpose_dropdown_2 = widgets.Dropdown(
    options=purposes,
    value=purposes[0],
    description="Purpose 2:"
)

period_dropdown_2 = widgets.Dropdown(
    options=periods,
    value=periods[0],
    description="Period 2:"
)

access_dropdown_2 = widgets.Dropdown(
    options=accesses,
    value=accesses[0],
    description="Access 2:"
)

# Function to update the chart based on selections
def update_chart(models_1, purpose_1, period_1, access_1, models_2, purpose_2, period_2, access_2):

    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:

        key1 = f"{purpose_1}_{access_1}_{period_1}_{models_1}"
        key2 = f"{purpose_2}_{access_2}_{period_2}_{models_2}"

        # Filter data for both selections
        filtered_df_1 = model_results[key1].copy()
        filtered_df_2 = model_results[key2].copy()

        
        if filtered_df_1.empty and filtered_df_2.empty:
            print("No data available for the selected criteria.")
            return
        
        filtered_df_1['key'] = 1
        filtered_df_2['key'] = 2

        global_min = min(filtered_df_1['cr_travel_distance'].min(), filtered_df_2['cr_travel_distance'].min())
        global_max = max(filtered_df_1['cr_travel_distance'].max(), filtered_df_2['cr_travel_distance'].max())

        bins = range(int(global_min), int(global_max) + 2, 5)

        fig, ax = plt.subplots(figsize=(10, 6))              

        ### Plotting the weighted distance frequency distribution
        sns.histplot(
            data=filtered_df_1,
            x='cr_travel_distance', 
            weights='trips_count', 
            bins=bins, 
            stat='density',
            ax=ax, 
            kde=True,
            kde_kws={"bw_adjust": 3.0},
            color='darkorange',
            label=key1,
            alpha=0.35)
        
        sns.histplot(
            data=filtered_df_2, 
            x='cr_travel_distance', 
            weights='trips_count', 
            bins=bins, 
            stat='density',
            ax=ax, 
            kde=True,
            kde_kws={"bw_adjust": 3.0},
            color='dodgerblue',
            label=key2,
            alpha=0.35)

        ### Adding labels and title
        ax.set_xlabel('CRT Distance')
        ax.set_ylabel('Frequency')
        ax.set_title(f"{key1} vs {key2}")

        ### Adding legend
        ax.legend(title='Legend')

        # Display the plot
        with output:
            plt.show() 
    
    else:
        firstTime = True


# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox1 = widgets.HBox([model_dropdown_1, purpose_dropdown_1, period_dropdown_1, access_dropdown_1])
hbox2 = widgets.HBox([model_dropdown_2, purpose_dropdown_2, period_dropdown_2, access_dropdown_2])
vbox = widgets.VBox([hbox1, hbox2])

# create interactive widget
interactive_output = widgets.interactive_output(update_chart, 
   {'models_1': model_dropdown_1, 
    'purpose_1': purpose_dropdown_1, 
    'period_1': period_dropdown_1, 
    'access_1': access_dropdown_1,
    'models_2': model_dropdown_2, 
    'purpose_2': purpose_dropdown_2, 
    'period_2': period_dropdown_2, 
    'access_2': access_dropdown_2})

display(vbox)
display(interactive_output)
display(output)

Output()

Output()